# chunker: default program

In [1]:
from default import *
import os

## Run the default solution on dev

In [4]:
chunker = LSTMTagger(os.path.join('data', 'train.txt.gz'), os.path.join('data', 'chunker'), '.tar')
decoder_output = chunker.decode('data/input/dev.txt')

100%|██████████| 1027/1027 [00:02<00:00, 459.66it/s]


## Evaluate the default output

In [5]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 11672 phrases; correct: 8568.
accuracy:  84.35%; (non-O)
accuracy:  85.65%; precision:  73.41%; recall:  72.02%; FB1:  72.71
             ADJP: precision:  36.49%; recall:  11.95%; FB1:  18.00  74
             ADVP: precision:  71.36%; recall:  39.45%; FB1:  50.81  220
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  70.33%; recall:  76.80%; FB1:  73.42  6811
               PP: precision:  92.40%; recall:  87.14%; FB1:  89.69  2302
              PRT: precision:  65.00%; recall:  57.78%; FB1:  61.18  40
             SBAR: precision:  84.62%; recall:  41.77%; FB1:  55.93  117
               VP: precision:  63.66%; recall:  58.25%; FB1:  60.83  2108


(73.40644276901988, 72.02420981842637, 72.70875763747455)

# Baseline

In [1]:
import sys
sys.path.append("..")
from chunker import *
import os

In [5]:
def character_level_repr(sentence):
    v_list = []

    for word in sentence:

        v1 = torch.zeros([width])
        v2 = torch.zeros([width])
        v3 = torch.zeros([width])

        if word is not '[UNK]':

            v1[printable_map[word[0]]] = 1
            v3[printable_map[word[-1]]] = 1

            for i in range(1, len(word) - 1):
                v2[printable_map[word[i]]] += 1

        v_cat = torch.cat([v1, v2, v3], 0)
        v_list.append(v_cat)
    return torch.stack(v_list)

In [6]:
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
decoder_output = chunker.decode('../data/input/dev.txt')

100%|██████████| 1027/1027 [00:07<00:00, 130.15it/s]


In [7]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('../data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 11930 phrases; correct: 9186.
accuracy:  86.95%; (non-O)
accuracy:  87.91%; precision:  77.00%; recall:  77.22%; FB1:  77.11
             ADJP: precision:  45.56%; recall:  18.14%; FB1:  25.95  90
             ADVP: precision:  68.38%; recall:  46.73%; FB1:  55.52  272
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  75.38%; recall:  80.52%; FB1:  77.87  6662
               PP: precision:  91.37%; recall:  88.45%; FB1:  89.88  2363
              PRT: precision:  70.27%; recall:  57.78%; FB1:  63.41  37
             SBAR: precision:  86.29%; recall:  45.15%; FB1:  59.28  124
               VP: precision:  69.06%; recall:  71.40%; FB1:  70.21  2382


(76.99916177703268, 77.21923335574982, 77.10904054394359)

## Analysis

We implemented the robust word recognition via semi-character RNN. The input vectors in semi-character RNN consists of three sub-vectors (v1, v2, v3) that correspond to the character positions where v1 is for the first character, v2 are for characters between the first and last characters, and v3 is for the last character. With this technique, we are able to treat the misspelt words as UNKNOWN words and do the robust phrasal chunking.

What we did here was to concatenate to the word embedding input to the chunker RNN an input vector that is the character level representation of the word, as it required in homework documents. After the character vectors concatenated with the word embeddings, the input dimension has changed from 128 to 428. We defined the above function character_level_repr() to implement the baseline.

After implementing the baseline, we got 77.1090 on dev test.


# Improvement Using Second RNN

In [1]:
import sys
sys.path.append("..")
from chunker import *
import os

In [2]:
class SecondRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(SecondRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.U = nn.Linear(embedding_dim, hidden_dim)
        
        self.tanh = nn.Tanh()

    def forward(self, sequence, hidden):
        hidden1 = self.tanh(self.W(hidden) + self.U(sequence))
        hidden2 = self.tanh(self.W(hidden1))

        return hidden2

In [3]:
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
decoder_output = chunker.decode('../data/input/dev.txt')

100%|██████████| 1027/1027 [00:12<00:00, 83.34it/s] 


In [4]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('../data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 12241 phrases; correct: 9581.
accuracy:  87.99%; (non-O)
accuracy:  88.94%; precision:  78.27%; recall:  80.54%; FB1:  79.39
             ADJP: precision:  48.57%; recall:  30.09%; FB1:  37.16  140
             ADVP: precision:  61.82%; recall:  54.52%; FB1:  57.94  351
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  77.86%; recall:  83.61%; FB1:  80.63  6698
               PP: precision:  93.03%; recall:  87.51%; FB1:  90.18  2296
              PRT: precision:  74.29%; recall:  57.78%; FB1:  65.00  35
             SBAR: precision:  73.83%; recall:  46.41%; FB1:  56.99  149
               VP: precision:  70.33%; recall:  78.52%; FB1:  74.20  2572


(78.26974920349645, 80.53967720242098, 79.38849069892694)

## Analysis

The improvement we try here was to create a second RNN that takes the character vectors created in baseline as input and use 2 hidden layers of this second RNN to get a temporary output. After that, we used the output to concatenate with the word embeddings to form the new input to the chunker RNN. The structure of RNN we implemented was stacked RNNs, where hidden states from RNN of a layer i are the inputs to RNN layer of a layer i+1.

Another thing we did here was to amplify the loss function by multiplying by a coefficient. At first, we chose 3 as the coefficient. Thus, the loss function should be "loss = loss_function(tag_scores, targets) * 3" which was shown in the Python code file.

Here, we got 79.3885 on dev test.

# Final version -- Modify Second RNN and Tune Loss Function

In [1]:
import sys
sys.path.append("..")
from chunker import *
import os

In [2]:
class SecondRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(SecondRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.U = nn.Linear(embedding_dim, hidden_dim)
        
        self.tanh = nn.Tanh()

    def forward(self, sequence, hidden):
        hidden1 = self.tanh(self.W(hidden) + self.U(sequence))
        #hidden2 = self.tanh(self.W(hidden1))

        return hidden1

In [3]:
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
decoder_output = chunker.decode('../data/input/dev.txt')

100%|██████████| 1027/1027 [00:13<00:00, 73.36it/s]


In [4]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('../data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 12376 phrases; correct: 9655.
accuracy:  88.07%; (non-O)
accuracy:  89.03%; precision:  78.01%; recall:  81.16%; FB1:  79.56
             ADJP: precision:  49.64%; recall:  30.53%; FB1:  37.81  139
             ADVP: precision:  62.22%; recall:  55.03%; FB1:  58.40  352
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  4
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  78.03%; recall:  83.74%; FB1:  80.78  6694
               PP: precision:  92.24%; recall:  88.57%; FB1:  90.37  2344
              PRT: precision:  63.89%; recall:  51.11%; FB1:  56.79  36
             SBAR: precision:  71.60%; recall:  51.05%; FB1:  59.61  169
               VP: precision:  69.67%; recall:  79.77%; FB1:  74.38  2638


(78.01389786683905, 81.16173503698722, 79.55669083717865)

## Analysis

We tried to tune the coefficient of the loss function and also improve the second RNN structure. Finally, we decided to use only one hidden layer in our second RNN and the coefficient we chose was 5, which means "loss = loss_function(tag_scores, targets) * 5" which is shown in the Python code file.

Finally, we got 79.5567 on dev test.